<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/california_housing/autokeras_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1
%pip install autokeras

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc1) to /tmp/pip-req-build-c57m_1bg
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-c57m_1bg
  Running command git checkout -q 0fb69434a132093518e0e53d40020145ae192629
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc1-cp36-none-any.whl size=85424 sha256=c06285553119fcabbdf312f3c56a7725966602f76610d57aae39a360351ba6f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-rqonbuf5/wheels/af/c9/7c/6ea01f9753a5dd1484136b4cb7b33a0a7fba253e5c74ade5af
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=2e3d5e02558bf7bcc77ca46588486f6fea14c7ef7159b2d15676769df3020071
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
     |████████████████████████████████| 122kB 2.8MB/s 


In [2]:
%pip show autokeras
%pip show keras-tuner
%pip show tensorflow

Name: autokeras
Version: 1.0.8
Summary: AutoML for deep learning
Home-page: http://autokeras.com
Author: Data Analytics at Texas A&M (DATA) Lab, Keras Team
Author-email: jhfjhfj1@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: packaging, pandas, scikit-learn, tensorflow
Required-by: 
Name: keras-tuner
Version: 1.0.2rc1
Summary: Hypertuner for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The Keras Tuner authors
Author-email: kerastuner@google.com
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: packaging, colorama, scikit-learn, future, tqdm, tabulate, requests, terminaltables, scipy, numpy
Required-by: 
Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: gast, wh

In [3]:
%cd /content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
%ls

/content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
autokeras_model.ipynb  auto_keras_outputs/  datasets/  fetch_data.ipynb


In [29]:
import cloudpickle, os, pprint

import pandas as pd
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score

import autokeras as ak
from autokeras import StructuredDataRegressor
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
F_COLS = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
TARGET_COL = 'MedHouseVal'

SEED = 7
MAX_TRIAL_NUM = 10
MODEL_OUTPUT_DIR = 'auto_keras_outputs'

EPOCH = 50
es = EarlyStopping(patience=3)

## 前処理していないデータで回帰モデルを構築

In [6]:
trial_name = 'origin_data_trial'

reg = StructuredDataRegressor(max_trials=MAX_TRIAL_NUM,
                              seed=SEED,
                              directory=os.path.join(MODEL_OUTPUT_DIR, trial_name),
                              overwrite=True)

In [7]:
%time
data_path = 'datasets/origin'
train_csv = 'train.csv'
val_csv = 'val.csv'

reg.fit(x=os.path.join(data_path, train_csv),
        y=TARGET_COL,
        validation_data=(os.path.join(data_path, val_csv), TARGET_COL),
        epochs=EPOCH,
        callbacks=[es],
        verbose=1)

Trial 10 Complete [00h 01m 34s]
val_loss: 0.30827993154525757

Best val_loss So Far: 0.30827993154525757
Total elapsed time: 00h 07m 07s
INFO:tensorflow:Oracle triggered exit


## 前処理済みのデータで回帰モデルを構築
行った前処理
- 標準化

In [8]:
std_trial_name = 'std_data_trial'

std_reg = StructuredDataRegressor(max_trials=MAX_TRIAL_NUM,
                                  seed=SEED,
                                  directory=os.path.join(MODEL_OUTPUT_DIR, std_trial_name),
                                  overwrite=True)

In [9]:
%time
std_data_path = 'datasets/std'
std_train_csv = 'std_train.csv'
std_val_csv = 'std_val.csv'

std_reg.fit(x=os.path.join(std_data_path, std_train_csv), 
            y=TARGET_COL,
            validation_data=(os.path.join(std_data_path, std_val_csv), TARGET_COL),
            epochs=EPOCH,
            callbacks=[es],
            verbose=1)

Trial 10 Complete [00h 01m 37s]
val_loss: 0.22447553277015686

Best val_loss So Far: 0.22447553277015686
Total elapsed time: 00h 11m 45s
INFO:tensorflow:Oracle triggered exit


## 回帰モデルの構造を比較

In [10]:
reg.export_model().summary()
std_reg.export_model().summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 8)                 0         
_________________________________________________________________
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense (Dense)                (None, 64)                576       
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

## モデルの評価

### 正解データの読み込み

In [11]:
%time

def _load_y_from_csv(csv_path, y_col_name):
  '''CSVファイルから予測対象の列のみ抽出'''
  df = pd.read_csv(csv_path, header=0)
  return df[y_col_name]


def load_y_from_csv_files(data_path, csv_names, y_col_name):
  '''CSVファイルリストから予測対象の列を抽出
  Parameters
  -----
  data_path : string
  csv_names : list of string
  y_col_name : string

  Returns
  -----
  data : dict
    {csv_name1:y_df2, csv_name2:y_df2, csv_name3:y_df3,,,,,}
  '''
  data = {}
  for csv_name in csv_names:
    csv_path = os.path.join(data_path, csv_name)
    data[csv_name] = _load_y_from_csv(csv_path, y_col_name)
  return data


test_csv = 'test.csv'
std_test_csv = 'std_test.csv'

origin_y = load_y_from_csv_files(data_path=data_path, csv_names=[train_csv, val_csv, test_csv],
                                 y_col_name=TARGET_COL)
std_y = load_y_from_csv_files(data_path=std_data_path, csv_names=[std_train_csv, std_val_csv, std_test_csv],
                              y_col_name=TARGET_COL)
pprint.pprint(origin_y)
pprint.pprint(std_y)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
{'test.csv': 0       3.60000
1       3.36000
2       2.69900
3       2.87500
4       5.00001
         ...   
4123    3.43700
4124    3.25900
4125    2.00000
4126    4.06500
4127    4.13700
Name: MedHouseVal, Length: 4128, dtype: float64,
 'train.csv': 0        1.285
1        1.875
2        0.525
3        0.913
4        2.860
         ...  
13204    3.305
13205    1.616
13206    1.210
13207    1.275
13208    1.696
Name: MedHouseVal, Length: 13209, dtype: float64,
 'val.csv': 0       0.863
1       1.485
2       1.335
3       1.149
4       1.405
        ...  
3298    1.739
3299    0.792
3300    0.559
3301    0.555
3302    0.875
Name: MedHouseVal, Length: 3303, dtype: float64}
{'std_test.csv': 0       1.353497
1       1.143981
2       0.566937
3       0.720582
4       2.575687
          ...   
4123    1.211201
4124    1.055809
4125   -0.043281
4126    1.759436
4127    1.822291
Name: MedHouseVal, Length: 4128, dtype: float64,
 

In [12]:
# 標準化器も読み込んでおく
def load_pkl(path):
  '''pklファイルの読み込み'''
  with open(path, 'rb') as f:
    obj = cloudpickle.load(f)
  return obj


pkl_name = 'std_scaler.pkl'
std_scaler = load_pkl(os.path.join(std_data_path, pkl_name))
std_scaler

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

### 評価指標の定義

In [48]:
def _calc_score(y_true, y_pred, sklearn_metric):
  '''sklearnの評価指標のスコアを計算'''
  score = sklearn_metric(y_true=y_true, y_pred=y_pred)
  return score


def evaluete_sk_metrics(y_true, y_pred, sklearn_metrics):
  '''sklearnの各種、評価関数を使用'''
  result = {}
  for sk_metric in sklearn_metrics:
    result[sk_metric.__name__] = _calc_score(y_true=y_true, y_pred=y_pred, sklearn_metric=sk_metric)
  return result

### 推論結果の取得

In [42]:
def predict_multiple_times(model, inputs, input_names):
  '''何度か推論'''
  pred = {}
  for input, name in zip(inputs, input_names):
    pred[name] = model.predict(x=input)
  return pred

In [45]:
%time
reg_inputs = [os.path.join(data_path, train_csv), os.path.join(data_path, val_csv),
              os.path.join(data_path, test_csv)]
reg_input_names = [train_csv, val_csv, test_csv]

std_reg_inputs = [os.path.join(std_data_path, std_train_csv), os.path.join(std_data_path, std_val_csv),
                  os.path.join(std_data_path, std_test_csv)]
std_input_names = [std_train_csv, std_val_csv, std_test_csv]


reg_preds = predict_multiple_times(model=reg, inputs=reg_inputs, input_names=reg_input_names)
std_reg_preds = predict_multiple_times(model=std_reg, inputs=std_reg_inputs, input_names=std_input_names)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


### モデルの評価の実行

In [67]:
SK_METRICS=[r2_score, explained_variance_score, mean_squared_error]

eval_scores = {trial_name: {}, std_trial_name: {}}

for csv_name in [train_csv, val_csv, test_csv]:
  eval_scores[trial_name][csv_name] = evaluete_sk_metrics(y_true=origin_y[csv_name],
                                                          y_pred=reg_preds[csv_name], sklearn_metrics=SK_METRICS)

for csv_name in [std_train_csv, std_val_csv, std_test_csv]:
  eval_scores[std_trial_name][csv_name] = evaluete_sk_metrics(y_true=std_y[csv_name],
                                                              y_pred=std_reg_preds[csv_name], sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'origin_data_trial': {'test.csv': {'explained_variance_score': 0.764497364700181,
                                    'mean_squared_error': 0.3172162537811974,
                                    'r2_score': 0.7641543761325529},
                       'train.csv': {'explained_variance_score': 0.7988240701046778,
                                     'mean_squared_error': 0.2641440815796903,
                                     'r2_score': 0.7986943587905473},
                       'val.csv': {'explained_variance_score': 0.778467302204158,
                                   'mean_squared_error': 0.3082799718189581,
                                   'r2_score': 0.7779319708087133}},
 'std_data_trial': {'std_test.csv': {'explained_variance_score': 0.7748218425954454,
                                     'mean_squared_error': 0.2313804887467392,
                                     'r2_score': 0.7742727121850517},
                    'std_train.csv': {'explained_variance_score': 0.808973

#### モデルの予測値を標準化から復元し評価する場合

In [96]:
# scalerの入力形状に合わせるためデータを読み込み
std_train_df = pd.read_csv(os.path.join(std_data_path, std_train_csv), header=0)
std_val_df = pd.read_csv(os.path.join(std_data_path, std_val_csv))
std_test_df = pd.read_csv(os.path.join(std_data_path, std_test_csv))


def inverse_scale_values(values, scaler):
  '''scalerを使って値の復元'''
  return scaler.inverse_transform(values)


inverse_scale_preds = {}
# 目的変数をモデルの予測値に置き換え
for df, csv_name, key in zip([std_train_df, std_val_df, std_test_df], [std_train_csv, std_val_csv, std_test_csv], 
                             [train_csv, val_csv, test_csv]):
  df[TARGET_COL] = std_reg_preds[csv_name]
  tmp_df = pd.DataFrame(inverse_scale_values(df.values, std_scaler), columns=F_COLS+[TARGET_COL])
  inverse_scale_preds[key] = tmp_df[TARGET_COL].values

inverse_scale_preds

{'test.csv': array([2.22010661, 2.97332405, 2.42588612, ..., 1.64286388, 3.44026607,
        4.34359164]),
 'train.csv': array([1.46262931, 1.90931729, 0.84499029, ..., 0.94900842, 1.64495533,
        1.61374175]),
 'val.csv': array([1.06052579, 0.81792568, 1.315066  , ..., 1.26904183, 0.93166764,
        1.30016598])}

In [98]:
# 標準化された値を復元した場合の評価スコア
for csv_name in [train_csv, val_csv, test_csv]:
  eval_scores[std_trial_name][csv_name] = evaluete_sk_metrics(y_true=origin_y[csv_name],
                                                              y_pred=inverse_scale_preds[csv_name],
                                                              sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'origin_data_trial': {'test.csv': {'explained_variance_score': 0.764497364700181,
                                    'mean_squared_error': 0.3172162537811974,
                                    'r2_score': 0.7641543761325529},
                       'train.csv': {'explained_variance_score': 0.7988240701046778,
                                     'mean_squared_error': 0.2641440815796903,
                                     'r2_score': 0.7986943587905473},
                       'val.csv': {'explained_variance_score': 0.778467302204158,
                                   'mean_squared_error': 0.3082799718189581,
                                   'r2_score': 0.7779319708087133}},
 'std_data_trial': {'std_test.csv': {'explained_variance_score': 0.7748218425954454,
                                     'mean_squared_error': 0.2313804887467392,
                                     'r2_score': 0.7742727121850517},
                    'std_train.csv': {'explained_variance_score': 0.808973